In [23]:
# working with polygons

import cv2
import numpy as np

# Create a callback function for mouse events
def create_polygon(event, x, y, flags, param):
    global image, polygon, mask

    if event == cv2.EVENT_LBUTTONDOWN:
        # Add the clicked point to the polygon
        polygon.append((x, y))

        # Draw lines between the polygon points on the image
        if len(polygon) > 1:
            cv2.line(image, polygon[-2], polygon[-1], (0, 0, 255), 2)
            cv2.imshow("Image", image)

        # Fill the mask with the polygon
        if len(polygon) > 2:
            mask.fill(0)
            points = np.array(polygon, np.int32)
            cv2.fillPoly(mask, [points], (255, 255, 255))

# Load the image
image = cv2.imread("Test/Input_RGBImage.png")

# Create a window to display the image
cv2.namedWindow("Image")
cv2.imshow("Image", image)

# Create an empty mask
mask = np.zeros_like(image)

# Create an empty polygon
polygon = []

# Set the callback function for mouse events
cv2.setMouseCallback("Image", create_polygon)

# Wait for key press
cv2.waitKey(0)

# Save the mask as a segmentation mask
cv2.imwrite("segmentation_mask.png", mask)

# Close all windows
cv2.destroyAllWindows()


In [24]:
# apply polygon to image

import cv2
import numpy as np

# Load the image
image = cv2.imread("Test/Input_RGBImage.png")

# Create a black mask of the same size as the image
mask = np.zeros_like(image)

# Define the polygon points
polygon_points = polygon

# Convert the polygon points to NumPy array
polygon_points = np.array(polygon_points)

# Draw the polygon on the mask
cv2.fillPoly(mask, [polygon_points], (255, 255, 255))

# Create a binary image where the polygon is white and the rest is black
binary_image = np.zeros_like(image)
binary_image[mask == 255] = 255

# Display the binary image

cv2.imshow("Binary Image", binary_image)

# save image as .npy file
np.save("binary_image.npy", binary_image)
#save image as .png file
cv2.imwrite("binary_image.png", binary_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [13]:
# open mask from Test/SegmentationMask
import cv2
import numpy as np

# Load the image
image = cv2.imread("Test/Input_ContentMaskClean.png")
ground_truth = cv2.imread("binary_image.png")

In [19]:
import cv2
import numpy as np

# Load the predicted mask
predicted_mask = cv2.imread("Test/Input_ContentMaskClean.png", cv2.IMREAD_GRAYSCALE)

# Load the ground truth mask
ground_truth_mask = cv2.imread("binary_image.png", cv2.IMREAD_GRAYSCALE)

# Convert the mask values to binary (0 or 255)
predicted_mask = (predicted_mask > 0).astype(np.uint8)
ground_truth_mask = (ground_truth_mask > 0).astype(np.uint8)

# Calculate the intersection and union of the masks
intersection = np.logical_and(predicted_mask, ground_truth_mask)
union = np.logical_or(predicted_mask, ground_truth_mask)

# Calculate the true positives, false positives, and false negatives
true_positive = np.sum(intersection)
false_positive = np.sum(predicted_mask) - true_positive
false_negative = np.sum(ground_truth_mask) - true_positive

# Calculate the precision, recall, and IoU
precision = true_positive / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
iou = true_positive / np.sum(union)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("IoU:", iou)


Precision: 0.9912539127232554
Recall: 0.8582702271821443
IoU: 0.8518196202531646


In [26]:
import cv2
import numpy as np
import os

# Function to create the polygon and save the mask
def create_polygon(image):
    # Create a window to display the image
    cv2.namedWindow("Image")
    cv2.imshow("Image", image)

    # Create an empty mask
    mask = np.zeros_like(image)

    # Create an empty polygon
    polygon = []

    # Create a callback function for mouse events
    def mouse_callback(event, x, y, flags, param):
        nonlocal image, polygon, mask

        if event == cv2.EVENT_LBUTTONDOWN:
            # Add the clicked point to the polygon
            polygon.append((x, y))

            # Draw lines between the polygon points on the image
            if len(polygon) > 1:
                cv2.line(image, polygon[-2], polygon[-1], (0, 0, 255), 2)
                cv2.imshow("Image", image)

            # Fill the mask with the polygon
            if len(polygon) > 2:
                mask.fill(0)
                points = np.array(polygon, np.int32)
                cv2.fillPoly(mask, [points], (255, 255, 255))

    # Set the callback function for mouse events
    cv2.setMouseCallback("Image", mouse_callback)

    # Wait for key press
    cv2.waitKey(0)

    # Close the window
    cv2.destroyAllWindows()

    return mask

# Set the path to the folder containing the images
folder_path = "Test"

# Iterate over the folders in the directory
for folder_name in os.listdir(folder_path):
    folder = os.path.join(folder_path, folder_name)

    # Skip files if any
    if not os.path.isdir(folder):
        continue

    # Set the image and mask paths
    image_path = os.path.join(folder, "Input_RGBImage.png")
    mask_path = os.path.join(folder, "segmentation_mask.png")

    # Load the image
    image = cv2.imread(image_path)

    # Create the polygon and save the mask
    mask = create_polygon(image)
    #cv2.imwrite(mask_path, mask)

    # Apply the polygon to the image
    binary_image = np.zeros_like(image)
    binary_image[mask == 255] = 255

    # Save the binary image as .npy and .png files
    binary_image_npy_path = os.path.join(folder, "segmentation_annotated.npy")
    binary_image_png_path = os.path.join(folder, "segmentation_annotated.png")
    np.save(binary_image_npy_path, binary_image)
    cv2.imwrite(binary_image_png_path, binary_image)


In [29]:
# load image from Test/SegmentationMask 
image = cv2.imread("Test/1/segmentation_mask.png")



In [30]:
image.max()

255

In [ ]:
# create test folder

import os

# Create a folder for the test images
os.makedirs("Test", exist_ok=True)

# Loop through volume_estimation/data/processed and copy the images to the test folder
for folder_name in os.listdir("volume_estimation/data/processed"):

    # if foldername contains Cell_Flask_160
    # take randomly ten folders containing Cell_Flask_160
    # copy the images to the test folder
    if folder_name.startswith("Cell_Flask_160"):

        # Create a folder for the images
        os.makedirs(os.path.join("Test", folder_name), exist_ok=True)

        # Loop through the images in the folder
        for image_name in os.listdir(os.path.join("volume_estimation/data/processed", folder_name)):
            # Copy the image to the test folder
            image_path = os.path.join("volume_estimation/data/processed", folder_name, image_name)
            new_image_path = os.path.join("Test", folder_name, image_name)
            shutil.copyfile(image_path, new_image_path)
